# UnSupervisedIOHMM

In [1]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

## Load data

In [2]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()

input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open', 'High', 'Low', 'Close']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close-next-day']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))



3259
tensor([1.5887, 1.4640, 1.2800, 1.0740, 1.0533])
tensor([[1.2667, 1.6667, 1.1693, 1.5927],
        [1.7193, 2.0280, 1.5533, 1.5887],
        [1.6667, 1.7280, 1.3513, 1.4640],
        [1.5333, 1.5400, 1.2473, 1.2800],
        [1.3333, 1.3333, 1.0553, 1.0740]])


tensor([  1.0000,   1.5887,   1.4640,  ..., 224.5700, 234.8600, 244.4000])

# Example 1

## Set up a simple model manully

In [3]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-6)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[-0.9880, -0.8880, -0.7954,  1.1781, -1.1204],
         [ 0.8260,  0.9800,  1.1449, -1.5280, -0.7040]],

        [[ 0.7336, -0.8079, -1.2494,  0.1465, -1.3178],
         [ 1.5204, -1.4786, -0.1438,  0.6324, -2.1494]]], requires_grad=True)
Parameter containing:
tensor([[ 0.1295, -0.2061,  0.6660,  1.3893,  1.6279],
        [ 0.8304,  0.3769,  1.0558,  1.1471, -0.9328]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [4]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[3.6315e-03, 9.9637e-01],
        [1.9516e-04, 9.9980e-01],
        [6.1931e-04, 9.9938e-01],
        [2.8570e-03, 9.9714e-01],
        [1.4082e-02, 9.8592e-01],
        [1.5729e-02, 9.8427e-01],
        [8.3607e-03, 9.9164e-01],
        [7.8502e-03, 9.9215e-01],
        [9.5679e-03, 9.9043e-01],
        [6.7634e-03, 9.9324e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [5]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[1.9516e-04, 9.9980e-01],
        [6.1931e-04, 9.9938e-01],
        [2.8570e-03, 9.9714e-01],
        [1.4082e-02, 9.8592e-01],
        [1.5729e-02, 9.8427e-01],
        [8.3607e-03, 9.9164e-01],
        [7.8502e-03, 9.9215e-01],
        [9.5679e-03, 9.9043e-01],
        [6.7634e-03, 9.9324e-01],
        [1.9874e-03, 9.9801e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [6]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[7.1146e-07, 1.0000e+00],
        [1.2097e-07, 1.0000e+00],
        [1.7755e-06, 1.0000e+00],
        [4.0922e-05, 9.9996e-01],
        [2.2820e-04, 9.9977e-01],
        [1.3471e-04, 9.9987e-01],
        [6.6705e-05, 9.9993e-01],
        [7.6430e-05, 9.9992e-01],
        [6.5777e-05, 9.9993e-01],
        [1.3560e-05, 9.9999e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [7]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[1.0447e-07, 5.0000e-01],
         [1.0447e-07, 5.0000e-01]],

        [[1.3748e-07, 5.0000e-01],
         [1.3748e-07, 5.0000e-01]],

        [[4.3770e-08, 5.0000e-01],
         [4.3770e-08, 5.0000e-01]],

        [[7.0984e-07, 5.0000e-01],
         [7.0984e-07, 5.0000e-01]],

        [[3.7712e-06, 5.0000e-01],
         [3.7712e-06, 5.0000e-01]],

        [[1.1238e-05, 4.9999e-01],
         [1.1238e-05, 4.9999e-01]],

        [[1.1738e-05, 4.9999e-01],
         [1.1738e-05, 4.9999e-01]],

        [[7.5380e-06, 4.9999e-01],
         [7.5380e-06, 4.9999e-01]],

        [[4.8798e-06, 5.0000e-01],
         [4.8798e-06, 5.0000e-01]],

        [[1.7304e-06, 5.0000e-01],
         [1.7304e-06, 5.0000e-01]]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [8]:
#IOHMM._baum_welch()

## See the training results

In [9]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[-0.9880, -0.8880, -0.7954,  1.1781, -1.1204],
         [ 0.8260,  0.9800,  1.1449, -1.5280, -0.7040]],

        [[ 0.7336, -0.8079, -1.2494,  0.1465, -1.3178],
         [ 1.5204, -1.4786, -0.1438,  0.6324, -2.1494]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[ 0.1295, -0.2061,  0.6660,  1.3893,  1.6279],
        [ 0.8304,  0.3769,  1.0558,  1.1471, -0.9328]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)


## Viterbi

In [10]:
#call the viterbi algorithm
IOHMM.viterbi()

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])